# Problema de Planejamento de Produção de Camisetas
  
Uma fábrica fará vendas nas quatro semanas de finais de um campeonato.  
- Demanda semanal: $d = [5000,\,10000,\,30000,\,60000]$.  
- Capacidade regular por semana: $P^{\max} = 25000$ camisetas.  
- Nas semanas 1 e 2 é possível contratar **hora extra** (até $O^{\max}=10000$ por semana) — custo por unidade em hora extra = R\$ 2,80.  
- Custo de produção por unidade (regulares): R\$ 2,00 nas semanas 1 e 2; R\$ 2,50 nas semanas 3 e 4 (devido à concorrência).  
- Custo de estocagem: $h = 0.20$ R\$ por unidade por semana.  
- Estoque inicial assumido zero. Excesso produzido pode ser estocado para atender demandas futuras.  
- Objetivo: **minimizar o custo total** (produção + horas extras + estoque) satisfazendo as demandas semanais.

---

## Modelagem Matemática

### Parâmetros
Sejam semanas $t = 1,2,3,4$.

- $d_t$: demanda na semana $t$.  
  $d_1=5000,\; d_2=10000,\; d_3=30000,\; d_4=60000$.

- $P^{\max}$: capacidade regular semanal.  
  $$ P^{\max} = 25000 $$

- $O^{\max}_t$: capacidade de hora extra na semana $t$:  
  $$ O^{\max}_1 = O^{\max}_2 = 10000, \quad O^{\max}_3 = O^{\max}_4 = 0 $$

- Custo unitário de produção regular por semana $c^{reg}_t$:  
  $$ c^{reg}_1 = c^{reg}_2 = 2.00, \quad c^{reg}_3 = c^{reg}_4 = 2.50 $$

- Custo unitário de hora extra (somente $t=1,2$): $c^{ot}_t = 2.80$.  
- Custo de estocagem por semana: $h = 0.20$.  
- Estoque inicial: $s_0 = 0$.

---

### Variáveis de decisão
Para cada semana $t$:

- $p_t \geq 0$: quantidade produzida usando **capacidade regular**.  
- $o_t \geq 0$: quantidade produzida em **hora extra**.  
- $s_t \geq 0$: estoque ao final da semana $t$.

---


### Função objetivo

Minimizar o custo total:
$$
\min Z = \sum_{t=1}^{4} \Big( c^{reg}_t \, p_t + c^{ot}_t \, o_t + h \, s_t \Big)
$$

---

### Restrições

**Balanço de estoque (para cada semana $t$):**
$$
s_{t-1} + p_t + o_t - d_t = s_t, \quad s_0 = 0
$$

**Capacidade regular:**
$$
0 \leq p_t \leq P^{\max}, \quad t=1,\dots,4
$$

**Limite de hora extra:**
$$
0 \leq o_t \leq O^{\max}_t, \quad t=1,\dots,4
$$

**Não-negatividade do estoque:**
$$
s_t \geq 0, \quad t=1,\dots,4
$$

---


In [2]:
# !pip install pulp

import pulp

# Parâmetros
weeks = [1,2,3,4]
d = {1:5000, 2:10000, 3:30000, 4:60000}
Pmax = 25000
Omax = {1:10000, 2:10000, 3:0, 4:0}
creg = {1:2.00, 2:2.00, 3:2.50, 4:2.50}
cot = {1:2.80, 2:2.80, 3:0.0, 4:0.0}  # cot unused for weeks 3,4 since Omax=0
h = 0.20
s0 = 0.0

# Modelo
model = pulp.LpProblem("Planejamento_Producao_Camisetas", pulp.LpMinimize)

# Variáveis
p = {t: pulp.LpVariable(f"p_{t}", lowBound=0, upBound=Pmax, cat="Continuous") for t in weeks}
o = {t: pulp.LpVariable(f"o_{t}", lowBound=0, upBound=Omax[t], cat="Continuous") for t in weeks}
s = {t: pulp.LpVariable(f"s_{t}", lowBound=0, cat="Continuous") for t in weeks}

# Função objetivo
model += pulp.lpSum([ creg[t]*p[t] + cot[t]*o[t] + h*s[t] for t in weeks ]), "Custo_total"

# Restrições de balanço de estoque
# s0 + p1 + o1 - d1 = s1, etc.
for t in weeks:
    if t == 1:
        model += s0 + p[1] + o[1] - d[1] == s[1], f"Balanco_semana_{t}"
    else:
        model += s[t-1] + p[t] + o[t] - d[t] == s[t], f"Balanco_semana_{t}"

# (As restrições de capacidade já estão aplicadas pelos bounds nas variáveis p e o)

# Resolver
model.solve(pulp.PULP_CBC_CMD(msg=1))

# Resultados
print("Status:", pulp.LpStatus[model.status])
print("Custo total (R$):", pulp.value(model.objective))
print("\nDecisões por semana:")
for t in weeks:
    print(f"Semana {t}: p={p[t].value():.0f}, o={o[t].value():.0f}, estoque final s={s[t].value():.0f}")



Status: Optimal
Custo total (R$): 258000.0

Decisões por semana:
Semana 1: p=25000, o=0, estoque final s=20000
Semana 2: p=25000, o=5000, estoque final s=40000
Semana 3: p=25000, o=0, estoque final s=35000
Semana 4: p=25000, o=0, estoque final s=0
